In [3]:
import numpy as np
import tensorflow as tf
import keras
import os
import sys
from tqdm import tqdm
import time
import keras.backend as K

2022-03-29 11:21:10.059206: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [4]:
sys.path.append('..')

In [238]:
import nets
from Data import datagen
import importlib 
import resnet1D_Ahmed
importlib.reload(nets)  # Python 3.4+

<module 'nets' from '/home/keondopark/sleep/JupyterNotebooks/../nets.py'>

In [6]:
np.random.seed(1)

In [7]:
PROCESSED_DATA_PATH = os.path.join('/home','aiot','data','origin_npy')
save_signals_path_SC = os.path.join(PROCESSED_DATA_PATH,'signals_SC_filtered')
save_annotations_path_SC = os.path.join(PROCESSED_DATA_PATH,'annotations_SC')
save_signals_path_ST = os.path.join(PROCESSED_DATA_PATH,'signals_ST_filtered')
save_annotations_path_ST = os.path.join(PROCESSED_DATA_PATH,'annotations_ST')

In [8]:
def match_annotations_npy(dirname, filepath):
    filename = os.path.basename(filepath)
    search_filename = filename.split('-')[0][:-2]
    file_list = os.listdir(dirname)
    filenames = [file for file in file_list if search_filename in file if file.endswith('.npy')]
    return filenames

In [147]:
dim_HT1D = (3000,1)
n_classes=6
epochs = 1
bs = 64
BASE_LEARNING_RATE = 1e-3
data_ratio = 0.25
PREV_CNT = 10
list_files_SC = [os.path.join(save_signals_path_SC, f) for f in os.listdir(save_signals_path_SC) if f.endswith('.npy')]
list_files_ST = [os.path.join(save_signals_path_ST, f) for f in os.listdir(save_signals_path_ST) if f.endswith('.npy')]

In [134]:
train_test_split = 0.7
split_cnt_SC = int(train_test_split * len(list_files_SC))
split_cnt_ST = int(train_test_split * len(list_files_ST))

list_files_train = []
list_files_test = []

list_ann_files_train = []
list_ann_files_test = []


list_files_SC_train = np.random.choice(list_files_SC[:split_cnt_SC], int(0.25 * split_cnt_SC), replace=False)
list_files_train += list_files_SC_train.tolist()
for f in list_files_SC_train:
    ann_file = match_annotations_npy(save_annotations_path_SC, f)
    list_ann_files_train.append(os.path.join(save_annotations_path_SC, ann_file[0]))

list_files_test += list_files_SC[split_cnt_SC:]

for f in list_files_SC[split_cnt_SC:]:
    ann_file = match_annotations_npy(save_annotations_path_SC, f)
    list_ann_files_test.append(os.path.join(save_annotations_path_SC, ann_file[0]))


list_files_ST_train = np.random.choice(list_files_ST[:split_cnt_ST], int(0.25 * split_cnt_ST), replace=False)
list_files_train += list_files_ST_train.tolist()
for f in list_files_ST_train:
    ann_file = match_annotations_npy(save_annotations_path_ST, f)
    list_ann_files_train.append(os.path.join(save_annotations_path_ST, ann_file[0]))

list_files_test += list_files_ST[split_cnt_ST:]
for f in list_files_ST[split_cnt_ST:]:
    ann_file = match_annotations_npy(save_annotations_path_ST, f)
    list_ann_files_test.append(os.path.join(save_annotations_path_ST, ann_file[0]))

train_generator = datagen.DataGenerator(list_files_train, list_ann_files_train, 
                          batch_size=bs, dim=dim_HT1D, n_classes=n_classes, shuffle=True)
test_generator = datagen.DataGenerator(list_files_test, list_ann_files_test, 
                          batch_size=bs, dim=dim_HT1D, n_classes=n_classes, shuffle=False)
   
train_generator2 = datagen.DataGenerator2(list_files_train, list_ann_files_train, 
                          batch_size=bs, dim=dim_HT1D, n_classes=n_classes, shuffle=True, prev_cnt=PREV_CNT)
test_generator2 = datagen.DataGenerator2(list_files_test, list_ann_files_test, 
                          batch_size=bs, dim=dim_HT1D, n_classes=n_classes, shuffle=False, prev_cnt=PREV_CNT)

In [135]:
from collections import defaultdict
cnt_class = defaultdict(int)
for x, y in train_generator:
    unique, counts = np.unique(y, return_counts=True)
    for i, cnt in zip(unique, counts):
        cnt_class[i] += cnt
cnt_class_np = np.zeros((n_classes,))
for i in range(n_classes):
    cnt_class_np[i] = cnt_class[i]
class_weight = 0.1 * np.ones((n_classes,))
class_weight[:n_classes-1] = sum(cnt_class_np[:n_classes-1])/(n_classes * cnt_class_np[:n_classes-1])

In [136]:
#array([49845.,  5164., 13992.,  2675.,  5113.,   139.])
cnt_class_np

array([49058.,  5067., 14266.,  3202.,  5844.,   126.])

In [137]:
np.sum(cnt_class_np)/64

1211.921875

In [150]:
len(list_ann_files_train), len(list_ann_files_test)

(32, 58)

In [151]:
list_ann_files_train

['/home/aiot/data/origin_npy/annotations_SC/SC4261FM-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4661EJ-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4722EM-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4641EP-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4091EC-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4742EC-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4021EH-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4451FY-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4532EV-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4042EC-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4571FV-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4502EM-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4401EC-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annotations_SC/SC4732EJ-Hypnogram.npy',
 '/home/aiot/data/origin_npy/annot

In [241]:
model = nets.Conv1DASPP_single()
model2 = nets.Conv1DASPP_multi(batch_size=bs, prev_cnt=PREV_CNT)

In [242]:
x = np.random.random((bs,3000,1))
x = tf.convert_to_tensor(x)
model(x)
model2(x)

<tf.Tensor: shape=(64, 6), dtype=float32, numpy=
array([[0.1711521 , 0.16680464, 0.15964754, 0.16473678, 0.17267556,
        0.16498333],
       [0.17335169, 0.16803941, 0.15796736, 0.1630061 , 0.17224145,
        0.165394  ],
       [0.17324673, 0.16878518, 0.16058475, 0.16374314, 0.17055361,
        0.16308664],
       [0.17326629, 0.16711335, 0.1595646 , 0.16380256, 0.17033017,
        0.16592304],
       [0.17190821, 0.1685011 , 0.15996855, 0.1631845 , 0.17161298,
        0.16482462],
       [0.17211924, 0.16867751, 0.15875745, 0.16469459, 0.17101847,
        0.16473275],
       [0.17242816, 0.16667183, 0.15910983, 0.16427645, 0.17245916,
        0.16505457],
       [0.1715452 , 0.16834572, 0.15970713, 0.16339435, 0.17142244,
        0.16558518],
       [0.17294171, 0.16811909, 0.15838183, 0.16340521, 0.17201154,
        0.16514057],
       [0.17245577, 0.16840912, 0.15901439, 0.16352877, 0.17146091,
        0.16513099],
       [0.1731981 , 0.16804264, 0.15891795, 0.16374621, 0.171

In [244]:
model.summary()

Model: "conv1daspp_single_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1daspp_layer_1 (Conv1DAS multiple                  3379456   
_________________________________________________________________
dense_27 (Dense)             multiple                  52230     
Total params: 3,431,686
Trainable params: 3,425,990
Non-trainable params: 5,696
_________________________________________________________________


In [245]:
def get_current_lr(epoch):
    lr = BASE_LEARNING_RATE
    for _ in range(epoch // 10):
        lr *= 0.1
    return lr

def adjust_learning_rate(optimizer, epoch):
    lr = get_current_lr(epoch)
    optimizer.learning_rate = lr

In [246]:
def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    weights = K.variable(weights)
        
    def loss(y_true, y_pred):
        bs = y_pred.shape[0]
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        #loss = -K.sum(loss, -1)
        loss = -K.sum(loss) / bs
        return loss
    
    return loss

In [247]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
#loss_fn = weighted_categorical_crossentropy(weights=class_weight)

In [248]:
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=optimizer, net=model)
manager = tf.train.CheckpointManager(ckpt, './ckpt_Advanced_Conv1D', max_to_keep=1)

In [249]:
start_epoch = 0
#if manager.latest_checkpoint:
#    ckpt.restore(manager.latest_checkpoint)
#    start_epoch = ckpt.step.numpy()-1
best_test_acc = 0.0

In [250]:
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        y_pred = model(x, training=True)
        loss_value = loss_fn(y, y_pred)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))    
    return loss_value, y_pred

@tf.function
def test_step(x, y):
    y_pred = model(x, training=False)
    return y_pred


In [251]:
for e in range(start_epoch, epochs):
    correct, total_cnt, total_loss = 0.0, 0.0, 0.0
    print('-'*20 + 'Epoch ' + str(e) + '-'*20)
    adjust_learning_rate(optimizer, e)
    start = time.time()
    for idx, (x, y) in enumerate(train_generator):   
        #print(x.shape)
        #loss, y_pred = train_step(x, y)
        with tf.GradientTape() as tape:
            y_pred = model(x, training=True)
            loss_value = loss_fn(y, y_pred)
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))    
        loss = loss_value

        total_cnt += y_pred.shape[0]
        y_pred_cls = tf.math.argmax(y_pred, axis=-1)
        correct += tf.reduce_sum(tf.cast(tf.equal(y_pred_cls, y), tf.float32))
        total_loss += loss * y_pred.shape[0]
        if (idx + 1) % 10 == 0 or idx+1 == len(train_generator):
            print("[%d / %d] Training loss: %.6f, Training acc: %.3f"%
                  (idx+1, len(train_generator), total_loss / total_cnt, correct / total_cnt),end='\r', flush=True)
        
    print("")
    print("Training loss: %.6f, Training acc: %.3f"%(total_loss / total_cnt, correct / total_cnt))
    print("Training time: %.2f sec "%(time.time() - start))
    ckpt.step.assign_add(1)
    
    if e+1 >= 10 and (e+1) % 5 == 0:
        start = time.time()
        
        correct, total_cnt, total_loss = 0.0, 0.0, 0.0
        for idx, (x, y) in enumerate(test_generator):
            #y_pred = model(x, training=False)
            
            y_pred = test_step(x, y)
            y_pred_cls = tf.math.argmax(y_pred, axis=-1)
            correct += tf.reduce_sum(tf.cast(tf.equal(y_pred_cls, y), tf.float32))
            total_cnt += y_pred.shape[0]
            y = tf.cast(y, dtype=tf.int32)
            y_onehot = tf.one_hot(y, depth=n_classes)
            total_loss += loss_fn(y, y_pred).numpy() * y_pred.shape[0]
            #total_loss += loss_fn(y_onehot, y_pred).numpy() * y_pred.shape[0]
                
            test_acc = correct / total_cnt
            test_loss = total_loss / total_cnt
            if (idx + 1) % 10 == 0 or idx+1 == len(test_generator):
                print("[%d / %d] test loss: %.6f, test accuracy: %.3f"%
                    (idx+1, len(test_generator), test_loss, test_acc),end='\r', flush=True)
            
        print("")
        print("test loss: %.6f, test acc: %.3f"%(test_loss, test_acc))
        print("Eval time: %.2f sec"%(time.time() - start))
        
        if test_acc > best_test_acc:
            best_test_acc = test_acc
            save_path = manager.save()
            print("Saved checkpoint for step {}: {}".format(int(ckpt.step), save_path))  

--------------------Epoch 0--------------------
[1206 / 1206] Training loss: 0.868147, Training acc: 0.805
Training loss: 0.868147, Training acc: 0.805
Training time: 174.43 sec 


In [262]:
model2.aspp.set_weights(model.aspp.get_weights())

In [263]:
model2.aspp.trainable=False

In [264]:
for e in range(start_epoch, epochs):
    correct, total_cnt, total_loss = 0.0, 0.0, 0.0
    print('-'*20 + 'Epoch ' + str(e) + '-'*20)
    adjust_learning_rate(optimizer, e)
    start = time.time()
    for idx, (x, y, batch_idx) in enumerate(train_generator2):  
        with tf.GradientTape() as tape:
            y_pred = model2(x, training=True)             
            loss_value = loss_fn(y, y_pred)
        grads = tape.gradient(loss_value, model2.trainable_weights)
        optimizer.apply_gradients(zip(grads, model2.trainable_weights))   
        
        total_cnt += y_pred.shape[0] - PREV_CNT
        y_pred_cls = tf.math.argmax(y_pred, axis=-1)
        correct += tf.reduce_sum(tf.cast(tf.equal(y_pred_cls[PREV_CNT:], y[PREV_CNT:]), tf.float32))
        if batch_idx == 0:
            correct += tf.reduce_sum(tf.cast(tf.equal(y_pred_cls[:PREV_CNT], y[:PREV_CNT]), tf.float32))
            total_cnt += PREV_CNT
        total_loss += loss * y_pred.shape[0]
        if (idx + 1) % 10 == 0 or idx+1 == len(train_generator2):
            print("[%d / %d] Training loss: %.6f, Training acc: %.3f"%
                  (idx+1, len(train_generator2), total_loss / total_cnt, correct / total_cnt),end='\r', flush=True)
        

--------------------Epoch 0--------------------


In [236]:
print("")
print("Training loss: %.6f, Training acc: %.3f"%(total_loss / total_cnt, correct / total_cnt))


Training loss: 0.217364, Training acc: 0.631


In [ ]:
correct, total_cnt, total_loss = 0.0, 0.0, 0.0
confusion_matrix = np.zeros((n_classes,n_classes))
for idx, (x, y) in enumerate(test_generator):
    y_pred = model(x, training=False)
    y_pred_cls = tf.math.argmax(y_pred, axis=-1)
    correct += tf.reduce_sum(tf.cast(tf.equal(y_pred_cls, y), tf.float32))
    total_cnt += y_pred.shape[0]
    y = tf.cast(y, dtype=tf.int32)    
    for i in range(n_classes):
        for j in range(n_classes):
            confusion_matrix[i,j] += np.sum((y_pred_cls.numpy()==i) * (y.numpy()==j))


In [ ]:
for i in range(n_classes):
    print_ln = ""
    for j in range(n_classes):
        print_ln += "%.3f "%(confusion_matrix[i,j] / np.sum(confusion_matrix[i]))
        #print_ln += "%d "%(confusion_matrix[i,j])
    print(print_ln)

In [ ]:
x = np.arange(10)
np.random.seed(1)
np.random.shuffle(x)

In [ ]:
x

In [ ]:
y = np.arange(20)
np.random.shuffle(y)

In [ ]:
y

In [ ]:
pc = np.array([[0,0,1],[1,0,0],[0,1,0], [3,4,5]])

In [ ]:
n = pc.shape[0]
x = np.tile(np.expand_dims(pc,1), [1,n,1])
y = np.empty((n,n,3))
y[:] = np.tile(np.expand_dims(pc,0), [n,1,1])
dist = np.sum((x - y) ** 2, axis=2) ** 0.5 # n by n matrix

In [ ]:
instances_list = []
thr = 2
for i in range(1, n):    
    included = False
    for inst_set in instances_list:
        if i in inst_set:
            included = True
            break
    if not included:
        close_pts = set()           
        q = [i]
        while q:
            j = q.pop()
            #if j in close_pts: continue
            new_pts = set(np.where(dist[j,:] < thr)[0])
            add_pts = new_pts - close_pts
            q += list(add_pts)
            close_pts = close_pts.union(add_pts)
        
        instances_list.append(close_pts)
centroids = []
for s in instances_list:
    cent = np.mean(pc[list(s),:], axis=0)
    centroids.append(cent)
    
    

In [ ]:
centroids, instances_list

In [ ]:
x = np.array([[[1,2,3,4],[5,6,7,8],[4,3,2,1]], [[1,2,3,4],[5,9,7,8],[4,3,2,1]]])

In [ ]:
x.shape

In [ ]:
x[:,:,0] == 5

In [ ]:
np.sum((x[:,:,0] == 5) * (x[:,:,1] == 6))

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
a = np.random.random((2,3000))
b = np.ones(2)
c = 'abc'

In [ ]:
df = pd.DataFrame(a)
df['b'] = b
df['c'] = c

In [ ]:
df

In [ ]:
d = np.random.random((2,3000))
e = np.ones(2)*2
f = 'def'
df2 = pd.DataFrame(d)
df2['b'] = e
df2['c'] = f

In [ ]:
df2

In [ ]:
df.append(df2)